In [1]:
train_df = pd.read_csv('/home/oem/sw/data/sum_train.csv')

train_df

NameError: name 'pd' is not defined

In [ ]:
train_real = train_df[train_df['label'] == 'real']
train_real

,id,path,label
3,AAAWBXQE,./train/AAAWBXQE.ogg,real
4,AAAWBXQE_PKNGFCHH_double,./fake_fake/AAAWBXQE_PKNGFCHH_double.wav,real
5,AABHDRLX,./train/AABHDRLX.ogg,real
6,AABHDRLX_MYTQQKLT_double,./fake_fake/AABHDRLX_MYTQQKLT_double.wav,real
7,AABXXHMU,./train/AABXXHMU.ogg,real
...,...,...,...
83014,ZZXEFKCS_YDSPLJMK_double,./fake_fake/ZZXEFKCS_YDSPLJMK_double.wav,real
83020,ZZYYUTZI,./train/ZZYYUTZI.ogg,real
83021,ZZYYUTZI_RXVBDTAP_double,./fake_fake/ZZYYUTZI_RXVBDTAP_double.wav,real
83024,ZZZJMLBU,./train/ZZZJMLBU.ogg,real


In [ ]:
train_real.to_csv('/home/oem/sw/data/train_real.csv', index=False, columns = ['id', 'path', 'label'])

In [ ]:
train_fake = train_df[train_df['label'] == 'fake']
train_fake

,id,path,label
0,AAACWKPZ,./train/AAACWKPZ.ogg,fake
1,AAACWKPZ_YDNYIBEK_double,./fake_fake/AAACWKPZ_YDNYIBEK_double.wav,fake
2,AAAQOZYI,./train/AAAQOZYI.ogg,fake
22,AADPDAER,./train/AADPDAER.ogg,fake
23,AADPDAER_HNXLBANC_double,./fake_fake/AADPDAER_HNXLBANC_double.wav,fake
...,...,...,...
83025,ZZZMJSBX,./train/ZZZMJSBX.ogg,fake
83026,ZZZRTSKN,./train/ZZZRTSKN.ogg,fake
83028,ZZZSYEYS,./train/ZZZSYEYS.ogg,fake
83029,ZZZSYEYS_LFNVGMHK_double,./fake_fake/ZZZSYEYS_LFNVGMHK_double.wav,fake


In [ ]:
train_fake.to_csv('/home/oem/sw/data/train_fake.csv', index=False, columns = ['id', 'path', 'label'])

In [1]:
import os
import torch
import torchaudio
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

class CustomDataset(Dataset):
    def __init__(self, csv_file, root_dir, max_length=10000, is_test=False):
        self.csv_file = csv_file
        self.root_dir = root_dir
        self.max_length = max_length
        self.is_test = is_test
        self.data = []
        self.labels = []
        self.ids = []
        self.load_data()

    def load_data(self):
        data = pd.read_csv(self.csv_file)

        for _, row in tqdm(data.iterrows(), total=len(data)):
            file_path = os.path.join(self.root_dir, 'test_wav' if self.is_test else 'train_wav', f"{row['id']}.wav")
            if os.path.exists(file_path):
                waveform, sample_rate = torchaudio.load(file_path)
                # print(sample_rate)
                # print(waveform.shape)
                # waveform = self.normalize_waveform(waveform)
            else:
                print(f"File not found: {file_path}")
                continue
            
            self.data.append(waveform)
            if not self.is_test:
                label = 1 if row['label'] == 'fake' else 0
                self.labels.append(label)
            self.ids.append(row['id'])
        
        print(f"Loaded {len(self.data)} samples")

    # def normalize_waveform(self, waveform):
    #     return (waveform - waveform.mean()) / waveform.std()
    
    def pad_waveform(self, waveform):
        if waveform.size(1) < self.max_length:
            padded_wav = torch.zeros((waveform.size(0), self.max_length))
            padded_wav[:, :waveform.size(1)] = waveform
        else:
            padded_wav = waveform[:, :self.max_length]
        return padded_wav
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        waveform = self.data[idx]
        waveform = self.pad_waveform(waveform)

        # (sequence_length) 형식으로 변환
        waveform = waveform.squeeze(0)
        sample = {'waveform': waveform, 'id': self.ids[idx]}
        if not self.is_test:
            sample['label'] = self.labels[idx]
        return sample


### Model Load


In [2]:
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor

# 모델 및 프로세서 로드
model_name_1 = "facebook/wav2vec2-base-960h"
model_name_2 = "facebook/wav2vec2-large-960h"
model_1 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_1, num_labels=1)
model_2 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_2, num_labels=1)
# processor = Wav2Vec2Processor.from_pretrained(model_name_1)

/home/oem/anaconda3/envs/sw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 

In [3]:
import torch

device = torch.device("cuda:1")
print(device)

cuda:1


### Real Train data Load

In [4]:
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

# 하이퍼파라미터 설정
epochs = 10
learning_rate = 1e-5

# 데이터셋 및 데이터로더 초기화
csv_file = '/home/oem/sw/data/train_real.csv'
root_dir = '/home/oem/sw/data'

In [5]:
train_dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

100%|██████████| 41367/41367 [01:09<00:00, 592.20it/s]

Loaded 41367 samples


### Loss Function

In [6]:
import torch.nn.functional as F

def info_nce_loss(feats, temperature=0.07):
    cos_sim = F.cosine_similarity(feats[:, None, :], feats[None, :, :], dim=-1)
    self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
    cos_sim.masked_fill_(self_mask, -1e5)  # 대각선 부분을 큰 음수로 채움
    pos_mask = self_mask.roll(shifts=cos_sim.shape[0] // 2, dims=0)
    cos_sim = cos_sim / temperature

    nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
    nll = nll.mean()

    return nll

### Real Model Training

In [7]:
# 옵티마이저 및 손실 함수 정의
optimizer_1 = AdamW(model_1.parameters(), lr=learning_rate)
optimizer_2 = AdamW(model_2.parameters(), lr=learning_rate)

model_1.to(device)
model_2.to(device)

# 모델 훈련
model_1.train()
model_2.train()
for epoch in range(epochs):
    total_loss_1 = 0.0
    total_loss_2 = 0.0
    for batch_idx, data in enumerate(tqdm(train_loader)):
        waveforms = data['waveform'].to(device)
        labels = data['label'].clone().detach().to(device)
        #print(waveforms.shape)
        
        # 입력 데이터 전처리
        #inputs = processor(waveforms, sampling_rate=16000, return_tensors="pt", padding=True)
        
        # print(waveforms.shape)

        # 모델 1 출력 계산 및 손실 계산
        outputs_1 = model_1(waveforms)
        loss_1 = info_nce_loss(outputs_1.logits)
        
        # 모델 2 출력 계산 및 손실 계산
        outputs_2 = model_2(waveforms)
        loss_2 = info_nce_loss(outputs_2.logits)
        
        # 역전파 및 옵티마이저 스텝
        optimizer_1.zero_grad()
        loss_1.backward()
        optimizer_1.step()
        
        optimizer_2.zero_grad()
        loss_2.backward()
        optimizer_2.step()
        
        total_loss_1 += loss_1.item()
        total_loss_2 += loss_2.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Model 1 Loss: {total_loss_1/len(train_loader)}, Model 2 Loss: {total_loss_2/len(train_loader)}")

100%|██████████| 5171/5171 [07:30<00:00, 11.47it/s]


Epoch 1/10, Model 1 Loss: 1.9850856695108776, Model 2 Loss: 1.9700105950809774


100%|██████████| 5171/5171 [07:36<00:00, 11.32it/s]


Epoch 2/10, Model 1 Loss: 1.9537087647095568, Model 2 Loss: 1.9858818035064871


100%|██████████| 5171/5171 [07:31<00:00, 11.44it/s]


Epoch 3/10, Model 1 Loss: 1.9681353983300909, Model 2 Loss: 2.0411156687969814


  3%|▎         | 152/5171 [00:13<07:21, 11.38it/s]


KeyboardInterrupt: 

In [8]:
torch.save(model_1.state_dict(), '/home/oem/sw/save_model/w2v_real_model3.pth')
torch.save(model_2.state_dict(), '/home/oem/sw/save_model/w2v_real_model4.pth')

print(f"Model saved to /home/oem/sw/save_model/w2v_real_model3.pth")
print(f"Model saved to /home/oem/sw/save_model/w2v_real_model4.pth")

Model saved to /home/oem/sw/save_model/w2v_real_model3.pth
Model saved to /home/oem/sw/save_model/w2v_real_model4.pth


In [9]:
model_1 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_1, num_labels=1)
model_2 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_2, num_labels=1)

model_1.load_state_dict(torch.load("/home/oem/sw/save_model/w2v_real_model1.pth"))
model_2.load_state_dict(torch.load("/home/oem/sw/save_model/w2v_real_model2.pth"))

model_1.to(device)
model_2.to(device)

model_1.eval()
model_2.eval()

print(f"Model_1 loaded from /home/oem/sw/save_model/w2v_real_model1.pth")
print(f"Model_2 loaded from /home/oem/sw/save_model/w2v_real_model2.pth")

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be

FileNotFoundError: [Errno 2] No such file or directory: '/home/oem/sw/save_model/w2v_real_model1.pth'

In [10]:
csv_file = '/home/oem/sw/data/test.csv'

test_dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, is_test=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

100%|██████████| 50000/50000 [03:25<00:00, 243.80it/s]

Loaded 50000 samples


In [11]:
predictions = []

with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(test_loader)):
        waveforms = data['waveform'].to(device)
        
        outputs_1 = model_1(waveforms)
        outputs_2 = model_2(waveforms)
        
        probs_1 = torch.sigmoid(outputs_1.logits).squeeze()
        probs_2 = torch.sigmoid(outputs_2.logits).squeeze()
        
        final_probs = (probs_1 + probs_2) / 2
        
        for i in range(final_probs.shape[0]):
            pred = {
                'id': data['id'][i],
                # 'fake': final_probs[i, 1].item(),
                'real': final_probs[i].item()
            }
            predictions.append(pred)

100%|██████████| 12500/12500 [09:56<00:00, 20.94it/s]


In [12]:
df_real_predictions = pd.DataFrame(predictions)
df_real_predictions

,id,real
0,TEST_00000,0.513379
1,TEST_00001,0.512487
2,TEST_00002,0.512406
3,TEST_00003,0.513182
4,TEST_00004,0.512507
...,...,...
49995,TEST_49995,0.511279
49996,TEST_49996,0.477292
49997,TEST_49997,0.513540
49998,TEST_49998,0.515091


In [13]:
df_real_predictions.to_csv('/home/oem/sw/submission/w2v_real.csv', index=False, columns = ['id', 'real'])

### Fake Train data Load

In [14]:
csv_file = '/home/oem/sw/data/train_fake.csv'
root_dir = '/home/oem/sw/data'

fake_train_dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir)
fake_train_loader = DataLoader(fake_train_dataset, batch_size=8, shuffle=True)

100%|██████████| 41664/41664 [00:56<00:00, 734.76it/s]

Loaded 41664 samples


In [15]:
# 옵티마이저 및 손실 함수 정의
optimizer_1 = AdamW(model_1.parameters(), lr=learning_rate)
optimizer_2 = AdamW(model_2.parameters(), lr=learning_rate)

model_1.to(device)
model_2.to(device)

# 모델 훈련
model_1.train()
model_2.train()
for epoch in range(epochs):
    total_loss_1 = 0.0
    total_loss_2 = 0.0
    for batch_idx, data in enumerate(tqdm(fake_train_loader)):
        waveforms = data['waveform'].to(device)
        labels = data['label'].clone().detach().to(device)
        #print(waveforms.shape)
        
        # 입력 데이터 전처리
        #inputs = processor(waveforms, sampling_rate=16000, return_tensors="pt", padding=True)
        
        # print(waveforms.shape)

        # 모델 1 출력 계산 및 손실 계산
        outputs_1 = model_1(waveforms)
        loss_1 = info_nce_loss(outputs_1.logits)
        
        # 모델 2 출력 계산 및 손실 계산
        outputs_2 = model_2(waveforms)
        loss_2 = info_nce_loss(outputs_2.logits)
        
        # 역전파 및 옵티마이저 스텝
        optimizer_1.zero_grad()
        loss_1.backward()
        optimizer_1.step()
        
        optimizer_2.zero_grad()
        loss_2.backward()
        optimizer_2.step()
        
        total_loss_1 += loss_1.item()
        total_loss_2 += loss_2.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Model 1 Loss: {total_loss_1/len(fake_train_loader)}, Model 2 Loss: {total_loss_2/len(fake_train_loader)}")

100%|██████████| 5208/5208 [20:20<00:00,  4.27it/s]


NameError: name 'train_loader' is not defined

In [16]:
print(f"Epoch {epoch+1}/{epochs}, Model 1 Loss: {total_loss_1/len(fake_train_loader)}, Model 2 Loss: {total_loss_2/len(fake_train_loader)}")


Epoch 1/100, Model 1 Loss: 1.9971224181296823, Model 2 Loss: 1.9524116807269611


In [17]:
torch.save(model_1.state_dict(), '/home/oem/sw/save_model/w2v_fake_model1.pth')
torch.save(model_2.state_dict(), '/home/oem/sw/save_model/w2v_fake_model2.pth')

print(f"Model saved to /home/oem/sw/save_model/w2v_fake_model1.pth")
print(f"Model saved to /home/oem/sw/save_model/w2v_fake_model2.pth")

Model saved to /home/oem/sw/save_model/w2v_fake_model1.pth
Model saved to /home/oem/sw/save_model/w2v_fake_model2.pth


In [18]:
model_1 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_1, num_labels=1)
model_2 = Wav2Vec2ForSequenceClassification.from_pretrained(model_name_2, num_labels=1)

model_1.load_state_dict(torch.load("/home/oem/sw/save_model/w2v_fake_model1.pth"))
model_2.load_state_dict(torch.load("/home/oem/sw/save_model/w2v_fake_model2.pth"))

model_1.to(device)
model_2.to(device)

model_1.eval()
model_2.eval()

print(f"Model_1 loaded from /home/oem/sw/save_model/w2v_fake_model1.pth")
print(f"Model_2 loaded from /home/oem/sw/save_model/w2v_fake_model2.pth")

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be

Model_1 loaded from /home/oem/sw/save_model/w2v_fake_model1.pth
Model_2 loaded from /home/oem/sw/save_model/w2v_fake_model2.pth


In [22]:
predictions = []

with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(test_loader)):
        waveforms = data['waveform'].to(device)
        
        outputs_1 = model_1(waveforms)
        outputs_2 = model_2(waveforms)
        
        probs_1 = torch.sigmoid(outputs_1.logits).squeeze()
        probs_2 = torch.sigmoid(outputs_2.logits).squeeze()
        
        final_probs = (probs_1 + probs_2) / 2
        
        for i in range(final_probs.shape[0]):
            pred = {
                'id': data['id'][i],
                'fake': final_probs[i].item(),
                # 'real': final_probs[i].item()
            }
            predictions.append(pred)

100%|██████████| 12500/12500 [10:13<00:00, 20.38it/s]


In [23]:
df_fake_predictions = pd.DataFrame(predictions)
df_fake_predictions

,id,fake
0,TEST_00000,0.404139
1,TEST_00001,0.407381
2,TEST_00002,0.406260
3,TEST_00003,0.405796
4,TEST_00004,0.405812
...,...,...
49995,TEST_49995,0.405429
49996,TEST_49996,0.408127
49997,TEST_49997,0.404169
49998,TEST_49998,0.405350


In [24]:
df_fake_predictions.to_csv('/home/oem/sw/submission/w2v_fake.csv', index=False, columns = ['id', 'fake'])

In [27]:
df_sum = pd.merge(df_fake_predictions, df_real_predictions,on='id',how='outer')

df_sum

,id,fake,real
0,TEST_00000,0.404139,0.513379
1,TEST_00001,0.407381,0.512487
2,TEST_00002,0.406260,0.512406
3,TEST_00003,0.405796,0.513182
4,TEST_00004,0.405812,0.512507
...,...,...,...
49995,TEST_49995,0.405429,0.511279
49996,TEST_49996,0.408127,0.477292
49997,TEST_49997,0.404169,0.513540
49998,TEST_49998,0.405350,0.515091


In [28]:
df_sum.to_csv('/home/oem/sw/submission/w2v_independent.csv', index=False, columns = ['id', 'fake', 'real'])